In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import RepeatVector
from keras.layers.wrappers import Bidirectional
from embeddings import Embeddings
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
embedding = Embeddings(100, 3, 1, 4)

In [3]:
word2vec_model = embedding.get_model()
vocab = embedding.get_vocabulary()

In [4]:
word2index = vocab[0]

In [5]:
contexts = ["""Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City.
He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices.
His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant.
His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
""","""Tesla went on to pursue his ideas of wireless lighting and electricity distribution in his high-voltage, high-frequency power experiments in New York and Colorado Springs, and made early (1893) pronouncements on the possibility of wireless communication with his devices. 
          He tried to put these ideas to practical use in an ill-fated attempt at intercontinental wireless transmission, his unfinished Wardenclyffe Tower project.
          In his lab he also conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.
          He also built a wireless controlled boat, one of the first ever exhibited."""]

In [6]:
max_len = max([len(word_tokenize(context)) for context in contexts])
"""tokenized_contexts = [word_tokenize(context) for context in contexts]
indexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]"""

'tokenized_contexts = [word_tokenize(context) for context in contexts]\nindexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]'

In [7]:
# padded_contexts = sequence.pad_sequences(indexed_contexts)

In [8]:
target = np.zeros(max_len*(max_len-1))

In [9]:
question = "In what year did Nikola Tesla emigrate to the United States?"

In [10]:
from itertools import product

In [11]:
context_question = list(product(contexts, [question]))

In [12]:
# tokenized_sent = [word_tokenize(sent) for sent in sent_tokenize(context)]

In [13]:
def get_average_sentence_vector(sentence):
    avg_sent_vec = np.zeros((100))
    count = 0
    for word in sentence:
        if word in word2vec_model.wv.vocab:
            avg_sent_vec += word2vec_model[word]
            count+=1
    avg_sent_vec = np.divide(avg_sent_vec, count)
    return avg_sent_vec

In [14]:
avg_sentences = []
for context, question in context_question:
    avg_sentences.append(get_average_sentence_vector(word_tokenize(context)))

In [15]:
avg_sentences = np.array(avg_sentences)

In [16]:
avg_question_vec = get_average_sentence_vector(word_tokenize(question))

In [17]:
x = np.array(list(product(avg_sentences, [avg_question_vec])))

In [18]:
x = x.reshape((2, 200))

In [19]:
sent_1_answer = target
sent_1_answer[13] = 1
sent_2_answer = target
y = np.array([sent_1_answer, sent_2_answer])

In [34]:
y = y.reshape(2, 1, 11130)

In [35]:
x = x.reshape(2,200)

In [37]:
model = Sequential()
model.add(Dense(100, input_shape=(200,)))
model.add(RepeatVector(1))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dense(y.shape[2], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
model.fit(x, y, epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 100)               20100     
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 1, 100)            0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 1, 200)            160800    
_________________________________________________________________
dense_26 (Dense)             (None, 1, 11130)          2237130   
Total params: 2,418,030
Trainable params: 2,418,030
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 1s - loss: 9.3439 - acc: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s - loss: 9.2947 - acc: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s - loss: 9.2390 - acc: 1.0000
Epoch 4

2/2 [==============================] - 0s - loss: 0.0045 - acc: 1.0000
Epoch 89/100
2/2 [==============================] - 0s - loss: 0.0044 - acc: 1.0000
Epoch 90/100
2/2 [==============================] - 0s - loss: 0.0044 - acc: 1.0000
Epoch 91/100
2/2 [==============================] - 0s - loss: 0.0044 - acc: 1.0000
Epoch 92/100
2/2 [==============================] - 0s - loss: 0.0043 - acc: 1.0000
Epoch 93/100
2/2 [==============================] - 0s - loss: 0.0043 - acc: 1.0000
Epoch 94/100
2/2 [==============================] - 0s - loss: 0.0043 - acc: 1.0000
Epoch 95/100
2/2 [==============================] - 0s - loss: 0.0042 - acc: 1.0000
Epoch 96/100
2/2 [==============================] - 0s - loss: 0.0042 - acc: 1.0000
Epoch 97/100
2/2 [==============================] - 0s - loss: 0.0042 - acc: 1.0000
Epoch 98/100
2/2 [==============================] - 0s - loss: 0.0042 - acc: 1.0000
Epoch 99/100
2/2 [==============================] - 0s - loss: 0.0041 - acc: 1.0000
Epoch

In [ ]:
model.predict(np.array([x[2]]))

In [ ]:
Bidirectional()